In [1]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HUB_OFFLINE"] = "1"
# os.environ["MAX_PIXELS"]=


In [2]:
import torch
import torch.nn.functional as F
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model_7b = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype="auto",
    #  attn_implementation="sdpa",
    #  device_map="auto",
).to("cuda:4")
model_2b = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    attn_implementation="eager",
    # device_map="auto",
).to("cuda:5")
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")
# processor_2b = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
from VLMEvalKit.vlmeval.vlm.qwen2_vl.qwen_mod_forward import (
    patch_forward,
    get_mean_attn_score,
    get_visual_token_mean_attn_score,
    get_visual_token_weight,
    calculate_dynamic_threshold,
)


In [7]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "You are a helpful assistant. Here is one image:"
            },
            {
                "type": "image",
                "image": "examples/image.png",
            },
            {
                "type": "text",
                "text": "Here is another image:"
            },
            {
                "type": "image",
                "image": "examples/sample.png",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
#print(image_inputs)


In [8]:
import numpy as np
from PIL import Image
from IPython.display import display

def generate_images(image_list, mode="noise", color=(255, 255, 255)):
    """
    根据 `mode` 生成与 `image_list` 中图像等大的随机噪声或纯色图像。

    参数：
    - image_list: list[PIL.Image]，输入的图像列表。
    - mode: str，"noise" 生成随机噪声图像，"blank" 生成纯色图像。
    - color: tuple，生成纯色图像时的颜色，默认为白色 (255, 255, 255)。

    返回：
    - list[PIL.Image]，生成的图像列表。
    """
    generated_images = []
    
    for img in image_list:
        width, height = img.size
        
        if mode == "noise":
            noise_array = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)
            generated_image = Image.fromarray(noise_array)
        elif mode == "blank":
            generated_image = Image.new("RGB", (width, height), color)
        else:
            raise ValueError("mode could only be 'noise' or 'blank'")
        
        generated_images.append(generated_image)
    
    return generated_images

neg_image_inputs = generate_images(image_inputs, mode="noise")


In [ ]:
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda:5")
output_ids = model_2b.generate(
            **inputs,
            return_dict_in_generate=True,
            output_attentions=True,
            max_new_tokens=128,
        )


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:5 and cuda:3! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
neg_inputs = processor(
    text=[text],
    images=neg_image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
neg_inputs = neg_inputs.to("cuda:3")
print(neg_inputs.pixel_values)
negative_output_ids = model_2b.generate(
            **neg_inputs,
            return_dict_in_generate=True,
            output_attentions=True,
            max_new_tokens=128,
        )


tensor([[-0.3032,  1.5508,  1.4632,  ..., -0.3000,  2.0321, -1.4376],
        [ 1.5800, -0.1134,  1.1712,  ...,  1.7762, -1.3949,  0.0840],
        [ 0.0763,  0.1055, -1.7485,  ..., -0.4279,  1.4349,  0.0982],
        ...,
        [ 0.0179,  1.6676,  0.1639,  ...,  0.5248, -0.7266, -1.3807],
        [ 0.4851, -1.7339,  0.5873,  ...,  1.2500, -0.9399,  2.1459],
        [ 1.9011,  0.4121,  0.5727,  ...,  1.5344,  1.1789,  0.1977]],
       device='cuda:3')


In [ ]:
# If you wanna check what's the output of negative generation, run me.

generated_ids = negative_output_ids.sequences
# generated_ids = output_ids.sequences

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
]
out = processor.tokenizer.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
response = out[0]
print(response)


The image appears to be a highly pixelated and distorted representation of a scene. The colors are vibrant and the pattern is chaotic, making it difficult to discern any specific details or objects within the image. The overall effect is one of visual noise and disorientation.


In [ ]:
aw = get_mean_attn_score(output_ids)

# neg_aw = get_mean_attn_score(negative_output_ids)
aw.shape


torch.Size([1276, 1276])

In [ ]:
vw = get_visual_token_mean_attn_score(
    aw,
    inputs,
    model_2b.config.vision_start_token_id,
    model_2b.config.vision_end_token_id,
)

# neg_vw = get_visual_token_mean_attn_score(
#     neg_aw,
#     neg_inputs,
#     model_2b.config.vision_start_token_id,
#     model_2b.config.vision_end_token_id,
# )
_ = [print(v.shape) for v in vw]


torch.Size([440])
torch.Size([666])


In [ ]:
vm_linear = [
  get_visual_token_weight(v, 0.6, 1.0, "linear", 0.0) for v in vw
]
model_7b.embed_weight = torch.concat(vm_linear, dim=0).to(model_7b.device)


In [ ]:
model_7b.forward = patch_forward.__get__(model_7b, Qwen2VLForConditionalGeneration)


In [ ]:
generated_ids = model_7b.generate(**inputs.to(model_7b.device), max_new_tokens=128)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
]
out = processor.tokenizer.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
response = out[0]


In [ ]:
print(response)
